## PyDSP를 이용한 DSP 연산 및 PyDSPTools를 이용한 데이터 시각화 Tutorial

### 파이썬 라이브러리 불러오기


In [4]:
# Data Import
import pydsptools.biorad as biorad

# Config
import pydsptools.config.pda as pda

# DSP processing
import pydsp.run.worker

# Analysis
import polars as pl
import pandas as pd

# Visualization
import pydsptools.plot as dspplt

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

### 1. PCR 데이터 준비 (Import)
DSP 연산을 하고자 하는 PCRD 파일의 Seegene Export 파일을 압축하여 준비한 후, 아래 정보를 입력함

* RAW_ZIP_PATH : PCRD 파일을 Seegene export 파일로 변환하여 압축한 파일의 경로
* TO_PARQUET_DIR : DSP 연산을 위해 Seegene export 파일을 paruqet으로 변환하여 저장할 디렉토리의 경로
* PCR_SYSTEM : 실험에서 사용한 PCR 기기 종류
* CONSUMABLE : 실험에서 사용한 소모품 종류 (8-strip, 96-film, 96-cap 중 선택)
* PROTOCOL : Quantstep 및 Temperature
* EXPERIMENT_NAME : 실험 이름
* PLATE_NAME : 플레이트 디자인 이름

In [23]:
RAW_ZIP_PATH = "./data/GI-B-I/GI-B-I-100/GI-B-I-100.zip"

PCR_SYSTEM = "CFX96"
CONSUMABLE = "8-strip"
PROTOCOL = {4: "Low", 5: "High"} # {QUANTSTEP: TEMPERATURE}

EXPERIMENT_NAME = "MY EXPERIMENT"
PLATE_NAME = "PLATE_NUM_001_100"

TO_PARQUET_DIR = "./data/GI-B-I/GI-B-I-100/computed/pcr_results"

위에서 입력한 정보와 함께 아래 명령어를 실행하면 TO_PARQUET_DIR에 PARQUET 파일들이 저장됨

In [24]:
(
    biorad.DataParser(RAW_ZIP_PATH, type="cfx-batch-csv")
    .append_metadata(
        pcr_system=PCR_SYSTEM,
        consumable=CONSUMABLE,
        protocol=PROTOCOL,
        experiment_name=EXPERIMENT_NAME,
        plate_name=PLATE_NAME,
    )
    .dump(TO_PARQUET_DIR)
)

PCR 파일들의 형태가 PARQUET으로 변환되어 DSP 연산 준비가 된 상태가 되었다는 것을 의미함 :)

### 2. 연산 설정 준비 (Config)

DSP 연산을 위한 설정값, 알고리즘을 정의하여 연산을 위한 Config 파일(YAML)을 생성하는 단계. 설정값, 알고리즘에 대한 정보를 아래 변수에 입력함

- CONFIG_NAME : Config 이름  
- DSP_VERSION : DSP 버전  
- DSP_PRODUCT : DSP 버전 별칭  
- COMMENTS : Config에 대한 부가 정보  
- XSLX_PATH_8_STRIP : 8_STRIP 설정값 파일 경로  
- CONFIG_YML_PATH : 저장한 YAML 파일 경로

In [ ]:
CONFIG_NAME = "tutorial_config"
DSP_VERSION = "2.1.1"
DSP_PRODUCT = "StrepAssay"
COMMENTS = "tutorial_config_using_mutliamp"

XSLX_PATH_8_STRIP = "data/config-xlsx/dspconfig_template_240305_multiamp.xlsx"

CONFIG_YML_PATH = "./config/tutorial_config.yml"

In [ ]:
(
    pda.ConfigParser("multiamp-2402", "xlsx")
        .set_dsp_version(DSP_VERSION)
        .set_dsp_product(DSP_PRODUCT)
        .set_name(CONFIG_NAME)
        .set_comments(COMMENTS)
        .add_config(CONSUMABLE, path=XSLX_PATH_8_STRIP)
        .dump(CONFIG_YML_PATH)
)

### 3. DSP 연산

Import에서 생성한 PCR Data parquet 파일과 연산 config 파일을 input으로 DSP 연산을 수행함

* INPUT_PARQUET_DIR : PCR 데이터 parquet 파일 디렉토리 경로
* CONFIG_YML_PATH : DSP 연산 CONFIG YAML 파일 경로
* TO_OUTPUT_DIR : DSP 연산 결과 parquet 파일 디렉토리 경로

In [25]:
INPUT_PARQUET_DIR = "./data/GI-B-I/GI-B-I-100/computed/pcr_results/"
CONFIG_YML_PATH = "./config/yaml/PRJDS001/GI-B-I/dsp2_generic_research_auto.yml"
TO_DSP_RESULT_DIR = "./data/GI-B-I/GI-B-I-100/computed/config__dsp2_generic_research_auto"

In [ ]:

pydsp.run.worker.multiple_tasks(
    INPUT_PARQUET_DIR, # Input directory
    CONFIG_YML_PATH,   # Configuration
    TO_DSP_RESULT_DIR,     # Output directory
    4,                 # Number of processes
    is_verbose=True    # Verboase mode
)

2024-04-19 00:31:52,559 - 109005 - INFO - Check entry /home/jupyter-kmkim/dsp-research-strep-a/kkm/data/GI-B-I/GI-B-I-100/computed/pcr_results/2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol).parquet
2024-04-19 00:31:52,559 - 109005 - INFO - Check entry /home/jupyter-kmkim/dsp-research-strep-a/kkm/data/GI-B-I/GI-B-I-100/computed/pcr_results/2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol).parquet
2024-04-19 00:31:52,559 - 109005 - INFO - Check entry /home/jupyter-kmkim/dsp-research-strep-a/kkm/data/GI-B-I/GI-B-I-100/computed/pcr_results/2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol).parquet
2024-04-19 00:31:52,559 - 109005 - INFO - Check entry /home/jupyter-kmkim/dsp-research-strep-a/kkm/data/GI-B-I/GI-B-I-100/computed/pcr_results/2. admin_2017-05-22 14-06-05_BR101459 GI9801XY MOM 소량 반제품1 민감도(Vpara, CdB, Vchol).parquet
2024-04-19 00:31:52,559 - 109005 - INFO - Check entry /home/jupyter-